In [4]:
import tweepy 
from decouple import config
from tweepy import OAuthHandler # to authenticate Twitter API
from tweepy import Stream 
from tweepy.streaming import StreamListener
import socket 
import json 

import tweepy
consumer_key =  config('consumer_key')
consumer_secret = config('consumer_secret')
access_token = config('access_token')
access_secret = config('access_secret')



In [5]:
import re
import tweepy
from tweepy import OAuthHandler
from textblob import TextBlob

class TwitterClient(object):
	'''
	Generic Twitter Class for sentiment analysis.
	'''
	def __init__(self):
		'''
		Class constructor or initialization method.
		'''
		# keys and tokens from the Twitter Dev Console
		consumer_key = config('consumer_key')
		consumer_secret = config('consumer_secret')
		access_token = config('access_token')
		access_token_secret = config('access_secret')

		# attempt authentication
		try:
			# create OAuthHandler object
			self.auth = OAuthHandler(consumer_key, consumer_secret)
			# set access token and secret
			self.auth.set_access_token(access_token, access_token_secret)
			# create tweepy API object to fetch tweets
			self.api = tweepy.API(self.auth)
		except:
			print("Error: Authentication Failed")

	def clean_tweet(self, tweet):
		'''
		Utility function to clean tweet text by removing links, special characters
		using simple regex statements.
		'''
		return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())

	def get_tweet_sentiment(self, tweet):
		'''
		Utility function to classify sentiment of passed tweet
		'''
		# create TextBlob object of passed tweet text
		analysis = TextBlob(self.clean_tweet(tweet))
		# set sentiment
		if analysis.sentiment.polarity > 0:
			return 'positive'
		elif analysis.sentiment.polarity == 0:
			return 'neutral'
		else:
			return 'negative'

	def get_tweets(self, query, count = 10):
		'''
		Main function to fetch tweets and parse them.
		'''
		# empty list to store parsed tweets
		tweets = []
		try:
			# call twitter api to fetch tweets
			fetched_tweets = self.api.search(q = query, count = count)
			# parsing tweets one by one
			for tweet in fetched_tweets:
				# empty dictionary to store required params of a tweet
				parsed_tweet = {}
				# saving text of tweet
				parsed_tweet['text'] = tweet.text
				# saving sentiment of tweet
				parsed_tweet['sentiment'] = self.get_tweet_sentiment(tweet.text)
				# appending parsed tweet to tweets list
				if tweet.retweet_count > 0:
					# if tweet has retweets, ensure that it is appended only once
					if parsed_tweet not in tweets:
						tweets.append(parsed_tweet)
				else:
					tweets.append(parsed_tweet)
			# return parsed tweets
			return tweets
		except tweepy.TweepError as e:
			# print error (if any)
			print("Error : " + str(e))

def main():
	# creating object of TwitterClient Class
	api = TwitterClient()
	# calling function to get tweets
	tweets = api.get_tweets(query = 'Donald Trump', count = 200)
	# picking positive tweets from tweets
	ptweets = [tweet for tweet in tweets if tweet['sentiment'] == 'positive']
	# percentage of positive tweets
	print("Positive tweets percentage: {} %".format(100*len(ptweets)/len(tweets)))
	# picking negative tweets from tweets
	ntweets = [tweet for tweet in tweets if tweet['sentiment'] == 'negative']
	# percentage of negative tweets
	print("Negative tweets percentage: {} %".format(100*len(ntweets)/len(tweets)))
	# percentage of neutral tweets
	print("Neutral tweets percentage: {} % \
		".format(100*(len(tweets) -(len( ntweets )+len( ptweets)))/len(tweets)))
	# printing first 5 positive tweets
	print("\n\nPositive tweets:")
	for tweet in ptweets[:10]:
		print(tweet['text'])
	# printing first 5 negative tweets
	print("\n\nNegative tweets:")
	for tweet in ntweets[:10]:
		print(tweet['text'])
if __name__ == "__main__":
	# calling main function
	main()


Positive tweets percentage: 39.62264150943396 %
Negative tweets percentage: 13.20754716981132 %
Neutral tweets percentage: 47.16981132075472 % 		


Positive tweets:
RT @Iyervval: Remember this? There was an America under a certain Donald Trump, who was supplying ventilators to India even when America it…
RT @MotherJones: Trump put a right-wing radio host in charge of a national park. Emails show the chaos that ensued. https://t.co/z0hRTFaqHb
RT @Bill_Maxwell_: Today would be a good day to indict Donald Trump.
RT @JENFL23: Rick Scott worked hard to get rich at the expense of the most vulnerable through the biggest Medicare fraud case in 🇺🇸 history…
RT @ewolf_7: Has Joe Biden been better than Donald Trump?

Yes           Yes          Yes
👇            👇            👇
@ABCNetwork @AndraDayMusic @USvsBillie Donald Trump won and y'all about to find out. Enjoy the show
RT @AaronParnas: Joe Biden has been in office for just over three months. 

He has already done more good for Americans than D

In [5]:
import findspark